In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
news_df = pd.read_csv('../../pre-analysis/datasets/news_with_sentiment.csv')

In [3]:
# Connect to Neo4j
uri = "bolt://localhost:7687"  
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [4]:
# Function to create nodes and relationships
def create_graph(tx, news_data):
    for _, row in news_data.iterrows():
        # Create Company Node
        tx.run("MERGE (c:Company {name: $company})", company=row['ticker'])

        # Create NewsArticle Node
        tx.run("""
            CREATE (n:NewsArticle {title: $title, date: $date, link: $link, sentiment: $sentiment})
            """, title=row['title'], date=row['date'], link=row['link'], sentiment=row['sentiment'])

        # Create Sentiment Node
        tx.run("MERGE (s:Sentiment {type: $sentiment})", sentiment=row['sentiment'])

        # Create Relationships
        tx.run("""
            MATCH (c:Company {name: $company}), (n:NewsArticle {title: $title}), (s:Sentiment {type: $sentiment})
            CREATE (n)-[:RELATED_TO]->(c)
            CREATE (n)-[:HAS_SENTIMENT]->(s)
            """, company=row['ticker'], title=row['title'], sentiment=row['sentiment'])


In [5]:
# Insert data into Neo4j
with driver.session() as session:
    session.write_transaction(create_graph, news_df)

/var/folders/wd/g8mvvv392p11yf16rzs6k9mr0000gn/T/ipykernel_68649/3218607630.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph, news_df)


In [6]:
print("Data successfully inserted into Neo4j.")
driver.close()

Data successfully inserted into Neo4j.
